# Genesis of Raphaël

Unleashing the power of DCNN to create random Raphaëls.

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Dropout, Input, Dense, Flatten, Reshape, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model

SAVE_PATH = "../../models"

In [68]:
def build_generator(latent_size):
    
    generator = Sequential([
        # (latent_size)
        Input(shape=(latent_size,), name="LatentNoise"),
        Dense(16*16*128, activation="relu"),
        # (16, 16, 384)
        Reshape((16, 16, 128)),
        # (32, 32, 192)
        UpSampling2D(),
        Conv2DTranspose(64, 3, padding="same", activation="relu"),
        BatchNormalization(momentum=0.8),
        # (64, 64, 96)
        UpSampling2D(),
        Conv2DTranspose(32, 3, padding="same", activation="relu"),
        BatchNormalization(momentum=0.8),
        # (128, 128, 3)
        UpSampling2D(),
        Conv2DTranspose(3, 3, padding="same", activation="tanh")
    ], name="generator")
    
    return generator

def build_discriminator():
    
    discriminator = Sequential([
        # (128, 128, 3)
        Input(shape=(128, 128, 3), name="InputImages"),
        Conv2D(32, 3, padding="same", kernel_initializer="he_normal"),
        LeakyReLU(0.2),
        Conv2D(32, 3, padding="same", kernel_initializer="he_normal"),
        LeakyReLU(0.2),
        MaxPooling2D(),
        Dropout(0.2),
        
        Conv2D(64, 3, padding="same", kernel_initializer="he_normal"),
        LeakyReLU(0.2),
        Conv2D(64, 3, padding="same", kernel_initializer="he_normal"),
        LeakyReLU(0.2),
        MaxPooling2D(),
        Dropout(0.2),
        
        Conv2D(128, 3, padding="same", kernel_initializer="he_normal"),
        LeakyReLU(0.2),
        Conv2D(128, 3, padding="same", kernel_initializer="he_normal"),
        LeakyReLU(0.2),
        MaxPooling2D(),
        
        Flatten(),
        Dropout(0.2),
        Dense(128, activation="relu"),
        Dropout(0.2),
        Dense(2, activation="softmax")        
    ], name="discriminator")
    
    return discriminator

In [69]:
generator = build_generator(100)

In [70]:
discriminator = build_discriminator()

In [71]:
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
leaky_re_lu_54 (LeakyReLU)   (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
leaky_re_lu_55 (LeakyReLU)   (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 64, 64, 64)      

In [72]:
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 32768)             3309568   
_________________________________________________________________
reshape_9 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_15 (UpSampling (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_transpose_27 (Conv2DT (None, 32, 32, 64)        73792     
_________________________________________________________________
batch_normalization_18 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
up_sampling2d_16 (UpSampling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_transpose_28 (Conv2DT (None, 64, 64, 32)        18

In [79]:
def build_combined(discriminator, generator, latent_size):
    noise = Input(shape=(latent_size, ), name="noise")
    fake_raph = generator(noise)
    discriminator.trainable = False
    validation = discriminator(fake_raph)
    return Model(noise, validation, name="combined")

In [80]:
combined = build_combined(discriminator, generator, 100)

In [81]:
combined.summary()

Model: "combined"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
noise (InputLayer)           [(None, 100)]             0         
_________________________________________________________________
generator (Sequential)       (None, 128, 128, 3)       3403075   
_________________________________________________________________
discriminator (Sequential)   (None, 2)                 4481698   
Total params: 7,884,773
Trainable params: 3,402,883
Non-trainable params: 4,481,890
_________________________________________________________________
